## Preparation

users need to first import the jupyterlab_nodeeditor and specify the address of the YAML files

In [1]:
import jupyterlab_nodeeditor as jlne

In [2]:
sample_3 = "./sample_models/sample_3.yml"
photosynthesis =  "./sample_models/photosynthesis.yml"

## Workspace

In [3]:
editor=jlne.yml_trans([sample_3, photosynthesis])
editor

AppLayout(children=(Label(value='Node Editor', layout=Layout(grid_area='header')), Accordion(layout=Layout(gri…

## Export the YAML file

In [7]:
jlne.editor_yaml(editor, "./sample_models/export_test.yml")

## Import Test

In [8]:
export_test="./sample_models/export_test.yml"

In [9]:
editor_test=jlne.yml_trans(export_test, show_instance=True)
editor_test

AppLayout(children=(Label(value='Node Editor', layout=Layout(grid_area='header')), Accordion(children=(VBox(ch…